In [100]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pprint

# Load dataset
df = pd.read_csv("sales_data.csv")

# Drop irrelevant/sensitive columns
df.drop(columns=["Product_ID", "Mobile", "EMAIL", "Location"], inplace=True)

# Drop missing values
df.dropna(inplace=True)

# ✅ Choose your regression target column (example: assume "Revenue")
# You must replace 'Revenue' with your actual regression target column.
target_column = "Converted"  # Replace with your actual numeric column
y = df[target_column]
X = df.drop(columns=[target_column])

# Categorical columns
categorical_columns = X.select_dtypes(include='object').columns.tolist()

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define regression models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42)
}

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(drop="first", handle_unknown='ignore'), categorical_columns)],
    remainder='passthrough'
)

# Evaluation
results = {}

for name, model in models.items():
    pipe = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    results[name] = {
        "MAE": mean_absolute_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "R² Score": r2_score(y_test, y_pred)
    }

# Pretty print results
pprint.pprint(results)


KeyError: 'Converted'

In [78]:
best_model = max(results, key=lambda x: results[x]["R2 Score"])
best_model,r2_score(y_test, y_pred)

('Gradient Boosting', -0.5380862275541494)